In [1]:
# autoreload module
# https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Import Network
import jax
from jax import numpy as jnp
from mtr.models.context_encoder.mtr_encoder import MTREncoder
from mtr.models.motion_decoder.bc_decoder import BCDecoder
from mtr.models.motion_decoder.q_decoder import QDecoder
from rl.actor import Actor
from rl.critic import Critic
from rl.encoder import Encoder
from rl.sac import SAC
import torch
import dataclasses
from waymax import config as _config
from waymax import datatypes
from waymax import dynamics
from rl_env.waymax_env import  MultiAgentEnvironment
from rl_env.env_utils import WomdLoader
import tensorflow as tf
import copy
from tqdm import tqdm
from mtr.config import cfg, cfg_from_yaml_file

2023-12-11 21:56:23.280018: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-11 21:56:23.280048: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-11 21:56:23.280051: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/home/zixuzhang/mambaforge/envs/waymax/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonote

In [3]:
from bc.bc_dataset import BCDataset, BCDatasetBuffer
from torch.utils.data import DataLoader
import torch

In [4]:
WOMD_1_2_0_TRAIN_LOCAL = _config.DatasetConfig(
    path='/Data/Dataset/Waymo/V1_2_tf/training/training_tfexample.tfrecord@1000',
    max_num_rg_points=30000,
    data_format=_config.DataFormat.TFRECORD,
    max_num_objects=32,
)

In [5]:
dataset = BCDatasetBuffer(WOMD_1_2_0_TRAIN_LOCAL, buffer_size=1000)
dataloader = DataLoader(dataset, batch_size=128, collate_fn=dataset.collate_fn, pin_memory=True, num_workers=0)

In [6]:
for i, data in tqdm(enumerate(dataloader)):
    if i==0:
        break

0it [00:00, ?it/s]No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


0it [00:04, ?it/s]


In [7]:
data.keys()

dict_keys(['obj_trajs', 'obj_trajs_mask', 'track_index_to_predict', 'obj_trajs_last_pos', 'map_polylines', 'map_polylines_mask', 'map_polylines_center', 'gt_action', 'scenario_id', 't'])